In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [2]:
df = pd.read_csv('sample_text.csv')
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [5]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [6]:
vectors.shape

(8, 768)

In [7]:
vectors

array([[-0.00247386,  0.03626738, -0.05290457, ..., -0.09152357,
        -0.03970006, -0.04330486],
       [-0.03357265,  0.0098052 , -0.03250131, ..., -0.05165471,
         0.02245886, -0.0315618 ],
       [-0.01865333, -0.04051305, -0.01235392, ...,  0.00610591,
        -0.07179645,  0.02773852],
       ...,
       [-0.00066453,  0.04252131, -0.05645506, ...,  0.0131547 ,
        -0.03183563, -0.04357663],
       [-0.03317152,  0.03252462, -0.02484841, ...,  0.01174419,
         0.0574712 ,  0.00571026],
       [-0.00166399,  0.00413823, -0.04597077, ...,  0.02008529,
         0.05656242, -0.00161591]], dtype=float32)

In [11]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [12]:
import faiss

index = faiss.IndexFlatL2(dim)


### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [13]:
index.add(vectors)

In [14]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000014AB2B46A00> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [15]:
search_query = "I want to buy a polo t-shirt"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [ ]:
vec

In [22]:
import numpy as np
reshapeVec = np.array(vec).reshape(1,-1)

In [23]:
reshapeVec.shape

(1, 768)

### Step 5: Search for similar vector in the FAISS index created

In [ ]:
distances, indices = index.search(reshapeVec,k=2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [ ]:
indices

array([[3, 2]], dtype=int64)

In [ ]:
indices.tolist()

[[3, 2]]

In [ ]:
row_indices = indices.tolist()[0]
row_indices

[3, 2]

In [29]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion
